In [1]:
import requests
import re
import json
import logging
from bs4 import BeautifulSoup as bs
'''
with open('/Users/daeyeop/Work/KBS Drama project/Data/drama_edited.json', 'r', encoding='utf-8') as f:
    drama = json.load(f)
    drama = set(drama)

with open('/Users/daeyeop/Work/KBS Drama project/Data/actor_edited.json', 'r', encoding='utf-8') as f:
    actors = json.load(f)
    actors = set(actors)
'''
base_url = 'https://search.naver.com/search.naver'
logging.basicConfig(filename='Drama.log', level=10, format='%(asctime)s:%(message)s')

def get_soup(url):
    html = requests.get(url)
    assert html.status_code == 200, 'Response needs to be 200, but {}'.format(html)
    soup = bs(html.content, 'html.parser')
    
    return soup


def get_cast_url(drama_url):
    soup = get_soup(drama_url)
    url_list = soup.find('ul',{'class':'tab_list'})
    for i in url_list.find_all('a'):
        try:
            url = i.attrs['href']
        except:
            logging.info('drama_url : {}, no attrs, in {}'.format(i, get_cast_url.__cast__))
            return None
        else:
            if 'cast%' in url:
                cast_url = base_url+url
    return cast_url


def isdrama(url):
    soup = get_soup(url)
    try:
        genre = soup.find('div', {'class':'sub_title'}).next.next.text
    except:
        return False
    if genre == '드라마':
        return True
    return False


def get_drama_urls(actor_url):
    
    pre_len_drama_set = len(drama)
    drama_url_list = []
    soup = get_soup(actor_url)
    result = soup.find('div',{'class':'workact_dsc'})
    try:
        drama_data = result.text.split('oPagingListUi76.displayResult(')[1].split('}]')[0]
    except:
        logging.info('actor_url : {}, no displayResult, in {}'.format(actor_url, get_drama_urls.__name__))
        return None
    for i in drama_data.split('\"onair_url\": \"'):
        if i.startswith('https://search.naver.com/search'):
            drama_url = i.split("\"")[0]
            if isdrama(drama_url) and not drama_url in drama:
                drama_url_list.append(drama_url) 
                drama.add(drama_url)
                print_title(drama_url)
    
    if len(drama_url_list) == 0:
        return None
    if pre_len_drama_set == len(drama):
        return None

    return drama_url_list


def get_actor_urls(cast_url):
    pre_len_actors_set = len(actors)
    actor_url_list = []
    people_search = 'https://people.search.naver.com/search.naver?where=nexearch&sm=tab_ppn&query={query}&os={os}&ie=utf8&key=PeopleService'
    soup = get_soup(cast_url)
    
    url_list = soup.find('div',{'class':"list_image_info"})
    url_list_parts = url_list.find_all('span',{'class':'sub_text type_ell_2 _html_ellipsis'})
    if (url_list_parts) == 0:
        logging.info('cast_url : {}, len is 0, in {}'.format(cast_url, get_actor_urls.__name__))
        return None
    for i in url_list_parts:
        try:
            url = i.a.attrs['href']
            url = url.split('&')
            for part in url:
                if part.startswith('os'):
                    os = part.split('=')[-1]
                elif part.startswith('query'):
                    query = part.split('=')[-1]
        except Exception as e:
            logging.info('actor_url : {}, error is {} in {}'.format(i, e, get_actor_urls.__name__))
            
        else:
            actor_url = people_search.format(query=query, os=os)
            
            if actor_url not in actors:
                actors.add(actor_url)
                actor_url_list.append(actor_url)
                print_title(actor_url)
        
    if len(actor_url_list) == 0:
        return None
    if pre_len_actors_set == len(actors):
        return None
    
    return actor_url_list

def save(google_drive=True, local_drive=True):
    
    drama_file = 'drama_edited.json'
    actor_file = 'actor_edited.json'
    
    if google_drive:
        with open('/Users/daeyeop/Google Drive File Stream/My drive/코어닷_드라마/{}'.format(drama_file), 'w', encoding='utf-8') as f:
            json.dump(list(drama), f, indent='\t')
        with open('/Users/daeyeop/Google Drive File Stream/My drive/코어닷_드라마/{}'.format(actor_file), 'w', encoding='utf-8') as f:
            json.dump(list(actors), f, indent='\t')
            
        print('drama : {}, actors : {} are saved in google_drive'.format(len(drama), len(actors)))
    
    if local_drive:
        with open('/Users/daeyeop/Work/KBS Drama project/Data/{}'.format(drama_file) , 'w', encoding='utf-8') as f:
            json.dump(list(drama), f, indent='\t')
        with open('/Users/daeyeop/Work/KBS Drama project/Data/{}'.format(actor_file), 'w', encoding='utf-8') as f:
            json.dump(list(actors), f, indent='\t')

    print('drama : {}, actors : {} are saved in local_drive'.format(len(drama), len(actors)))

def crawler(actor_url):
    drama_url_list = get_drama_urls(actor_url)  ### drama.add(drama_urls)
    if drama_url_list is None:
        return None
    for drama_url in drama_url_list:
        cast_url = get_cast_url(drama_url)
        actor_url_list = get_actor_urls(cast_url) ### actors.add(actor_url)
        print(drama_url)
        if actor_url_list is None:
            return None
        for actor_url in actor_url_list:
            crawler(actor_url)
            save()
            
def print_title(url):
    soup = get_soup(url)
    print(soup.title.text)
             

In [2]:
def drama_data(actor_url):
    soup = get_soup(actor_url)
    name = soup.title.text.split()[0]
    try:
        text = soup.find('ul', {'id':"listUI_76"}).parent.next_sibling.next.text
        text = text.split('setOnairInfo(contentsid)')[1]
    except:
        print('Check {} : {}'.format(name, actor_url))
        return None
    characterLink = text.split('characterLink = \'')
    drama_characters = {}
    for i in text.split('characterName')[1:]:
        
        if 'http://search.naver.com/search' in i:
            data_parts = i.split(';')
            character = data_parts[0].split()[-1].strip('\'')
            character_url = data_parts[1].split()[-1].strip('\'')
            role = data_parts[2].split()[-1]
            role = re.sub('[\(\)\']', '', role)
            soup = get_soup(character_url)
            drama_title = get_drama_title(character, soup)
            character_desc = get_character_desc(soup)
            
            drama_characters[drama_title] = {'role' : role,
                                             'desc' : character_desc
                                            }

    
    drama_data_dict[name] = {'drama' : drama_characters, 'actor_url' : actor_url}
    return drama_data_dict

In [5]:
def get_drama_title(character, soup):
    title = soup.title.text.split(character)[0]
    return title

def get_character_desc(soup):
    try:
        desc = soup.find('span',{'class':'desc _text'}).text
    except:
        return None
    return desc

In [11]:
with open('/Users/daeyeop/Work/KBS Drama project/Data/drama_actors.json', 'r', encoding='utf-8') as f:
    drama_data_dict = json.load(f)

In [66]:
drama_data_dict

{'박정학': {'drama': {'루갈': {'role': '조연',
    'desc': '아르고스의 전신인 불개미파 보스. 2008년 금융위기 속 내수 부양을 위해 지하 자본을 양성하려는 정치권과 결탁, 세를 확장해 자본력을 확보했고 아르고스라는 합법적인 브랜드를 탄생시켰다. 10여 년이 흘러 아르고스는 마약, 매춘, 유흥 등 음지뿐 아니라 유통, 엔터테인먼트 등 양지의 사업에까지 손을 뻗은, 대한민국의 최대 범죄 조직이자 테러 집단으로 컸다.'},
   '싸이코패스다이어리': {'role': '조연',
    'desc': '60대, 지훈을 후계자로 생각하고는 있지만, 한참 멀었다고 생각하고 이래저래 인우보다 한 수 아래라 생각한다. 인우를 그렇게 높게 평가하면서도, 서자인 탓에 야박하고 엄하게 대한다. 사실은 어려서부터 인우가 ‘남들과 다르다는 것’을 유일하게 인지했다.'},
   '더뱅커': {'role': '조연',
    'desc': '서민경제 정책 연구소의 실질적 보스. 감으로 때려잡고 정보원 관리하는 구시대적 수사법이 문제가 되어 사직하게 된 전직 경찰 출신. 과거 형사 시절 자신이 붙잡았던 문제아 재벌 4세들(더 베스트)의 손과 발이 되어 더러운 일을 한다.'},
   '용왕님보우하사': {'role': '조연',
    'desc': '주보그룹의 가신. 어제의 잘못을 오늘의 잘못으로 끝없이 덮어나가는 인물. 주보그룹과 관련된 이들에게 닥친 모든 불행의 배후에는 그가 있다. 목표는 오로지 자신의 욕망을 실현하는 것일 뿐. 일말의 죄책감이나 후회 따위 없는 경주마 같은 사내. 때로는 비굴한 아첨으로, 때로는 서늘한 위협으로 자신이 원하는 모든 걸 가지고야 마는 무서운 집착과 소유욕을 지녔다.'},
   '돈꽃': {'role': '조연',
    'desc': '오기평. 정말란의 그림자이자 개인 기사. 청아가 대저택의 살림을 책임지는 총무요, 집사인데 말란의 개인 기사 노릇을 하기 때문에 모두 오기사라 부른다. 말란의 친정에서 대를 이어 기

In [65]:
'''
for actor_url in list_actors:
    try:
        drama_data(actor_url)
    except:
        print('Failed, ',actor_url)
    else:
        with open('/Users/daeyeop/Google Drive File Stream/My drive/코어닷_드라마/drama_actors.json', 'w', encoding='utf-8') as f:
            json.dump(drama_data_dict, f, indent='\t')
        with open('/Users/daeyeop/Work/KBS Drama project/Data/drama_actors_v2.json', 'w', encoding='utf-8') as f:
            json.dump(drama_data_dict, f, indent='\t', ensure_ascii=False)
'''

"\nfor actor_url in list_actors:\n    try:\n        drama_data(actor_url)\n    except:\n        print('Failed, ',actor_url)\n    else:\n        with open('/Users/daeyeop/Google Drive File Stream/My drive/코어닷_드라마/drama_actors.json', 'w', encoding='utf-8') as f:\n            json.dump(drama_data_dict, f, indent='\t')\n        with open('/Users/daeyeop/Work/KBS Drama project/Data/drama_actors_v2.json', 'w', encoding='utf-8') as f:\n            json.dump(drama_data_dict, f, indent='\t', ensure_ascii=False)\n"

In [13]:
for failed_url in failed_urls:
    drama_data(failed_url)

In [15]:
with open('/Users/daeyeop/Google Drive File Stream/My drive/코어닷_드라마/drama_actors.json', 'w', encoding='utf-8') as f:
        json.dump(drama_data_dict, f, indent='\t')
with open('/Users/daeyeop/Work/KBS Drama project/Data/drama_actors_v2.json', 'w', encoding='utf-8') as f:
        json.dump(drama_data_dict, f, indent='\t', ensure_ascii=False)